# Introdução

Este é o notebook em que vários testes de redes neurais com diferentes parâmetros podem ser rodados para que se tenha uma comparação justa para treinar uma rede única final que utilizará os dados de teste final para comparar se o método foi realmente efetivo

# Imports

Começamos importando as funções, bibliotecas e módulos necessários para fazer a rede neural e avaliar seus resultados

In [1]:
# Funções criadas para pegar os dados e criar cada mlp separadamente
from funcoes import get_filtered_df, get_additive_input_type_1, get_additive_input_type_2, cria_mlp

# Módulo para criação de DataFrames
import pandas as pd
# Módulo para carregar dados de arquivos .pickle (python)
import pickle

# Módulo torch para referênciação das funções de ativação
import torch

# Módulo para divisão entre dados de treino e teste (treino, teste e teste real)
from sklearn.model_selection import train_test_split
# Módulo para normalização dos dados para manter um padrão de valores
from sklearn.preprocessing import StandardScaler

# DataFrame

Com as funções, bibliotecas e módulos importados, podemos pegar os dados da tabela de excel obtida no banco de dados. Para mais informações acessar o github: https://github.com/selaxco/Melting-Point-Prediction

In [2]:
df = get_filtered_df('Dados/Human (Jurkat lymphoma T cells).xlsx', 300)
df

,index,PROTEIN,LENGTH,UNIPROT_ID,Tm_(C),ADDITIVES
0,2,GTPase NRas,189.0,P01111,62.18,NaN
1,6,Rho-related GTP-binding protein RhoB,196.0,P62745,51.06,NaN
2,7,Prefoldin subunit 1,122.0,O60925,56.02,NaN
3,11,Ribonuclease H1,286.0,O60930,43.11,NaN
4,17,Short coiled-coil protein,159.0,Q9UIL1,53.87,NaN
...,...,...,...,...,...,...
5951,28186,Gamma-glutamylaminecyclotransferase,153.0,Q9BVM4,51.63,0.67% NP40 (octylphenoxypoly(ethyleneoxy)ethanol)
5952,28220,Prostamide/prostaglandin F synthase,198.0,Q8TBF2,54.11,0.67% NP40 (octylphenoxypoly(ethyleneoxy)ethanol)
5953,28276,NEDD8-conjugating enzyme UBE2F,185.0,Q969M7,53.66,0.67% NP40 (octylphenoxypoly(ethyleneoxy)ethanol)
5954,28352,Rab-like protein 2A,228.0,Q9UBK7,48.03,0.67% NP40 (octylphenoxypoly(ethyleneoxy)ethanol)


# Aminoacidos

Podemos então carregar os dados dos aminoácidos já calculados por outro notebook

In [3]:
file = 'Dados/proteins_aminoacids.pickle'

with open(file, 'rb') as file_handler:
    proteins = pickle.load(file_handler)

i = 0
for protein in proteins:
    output = f'ID: {protein}\n'
    for amino in proteins[protein]:
        output += f'{amino:1}:{proteins[protein][amino]:2} \ '
    print(output[:-2])
    i += 1
    if i >= 10:
        break

ID: P01111
A:10 \ C: 5 \ D:16 \ E:12 \ F: 6 \ G:14 \ H: 2 \ I:11 \ K:12 \ L:14 \ M: 7 \ N: 6 \ P: 5 \ Q:10 \ R:10 \ S:10 \ T:13 \ V:17 \ W: 0 \ Y: 9 
ID: P62745
A:15 \ C: 8 \ D:16 \ E:16 \ F: 6 \ G: 9 \ H: 2 \ I: 9 \ K:14 \ L:15 \ M: 4 \ N: 6 \ P:10 \ Q: 6 \ R:12 \ S: 8 \ T: 9 \ V:22 \ W: 2 \ Y: 7 
ID: O60925
A:11 \ C: 0 \ D: 6 \ E:16 \ F: 2 \ G: 2 \ H: 3 \ I: 9 \ K:15 \ L:13 \ M: 6 \ N: 3 \ P: 1 \ Q:10 \ R: 6 \ S: 4 \ T: 7 \ V: 6 \ W: 0 \ Y: 2 
ID: O60930
A:28 \ C: 5 \ D:12 \ E:21 \ F:13 \ G:27 \ H: 9 \ I:10 \ K:17 \ L:15 \ M: 7 \ N:12 \ P:14 \ Q:11 \ R:24 \ S:17 \ T:12 \ V:18 \ W: 8 \ Y: 6 
ID: Q9UIL1
A:10 \ C: 2 \ D: 9 \ E:12 \ F: 5 \ G: 8 \ H: 2 \ I: 3 \ K: 9 \ L:17 \ M: 6 \ N: 6 \ P: 6 \ Q:10 \ R:15 \ S:18 \ T: 6 \ V:11 \ W: 2 \ Y: 2 
ID: Q9P086
A:15 \ C: 2 \ D: 5 \ E:12 \ F: 1 \ G: 4 \ H: 2 \ I: 5 \ K: 5 \ L:14 \ M: 3 \ N: 4 \ P: 1 \ Q: 8 \ R:10 \ S: 9 \ T: 7 \ V: 6 \ W: 0 \ Y: 4 
ID: Q9BX68
A:25 \ C: 1 \ D: 8 \ E: 6 \ F: 3 \ G:18 \ H: 5 \ I: 8 \ K: 7 \ L:18 \ M: 1 \ N: 2 \ P:10 

# Tratando dados

Podemos tratar os dados para que fiquemos apenas com nossas features e targets, além de informações importantes

In [4]:
dados = []

for protein, id_, additive, melting_point in zip(df['PROTEIN'], df['UNIPROT_ID'], df['ADDITIVES'], df['Tm_(C)']):
    semi_dados = [protein]
    
    for amino in proteins[id_]:
        quantity_amino = proteins[id_][amino]
        semi_dados.append(quantity_amino)
    
    additive_type1 = get_additive_input_type_1(additive)
    additive_type2 = get_additive_input_type_2(additive)
    
    semi_dados.append(additive_type1)
    semi_dados.append(additive_type2[0])
    semi_dados.append(additive_type2[1])
    semi_dados.append(melting_point)

    dados.append(semi_dados)

columns = ['Nomes']
columns += [amino for amino in 'ACDEFGHIKLMNPQRSTVWY']
columns += ['Type1 Additive', 'Type2 Additive 1', 'Type2 Additive 2', 'Melting Point (C)']

df_dados = pd.DataFrame(dados, columns=columns)
df_dados

,Nomes,A,C,D,E,F,G,H,I,K,...,R,S,T,V,W,Y,Type1 Additive,Type2 Additive 1,Type2 Additive 2,Melting Point (C)
0,GTPase NRas,10,5,16,12,6,14,2,11,12,...,10,10,13,17,0,9,0.000000,0,0,62.18
1,Rho-related GTP-binding protein RhoB,15,8,16,16,6,9,2,9,14,...,12,8,9,22,2,7,0.000000,0,0,51.06
2,Prefoldin subunit 1,11,0,6,16,2,2,3,9,15,...,6,4,7,6,0,2,0.000000,0,0,56.02
3,Ribonuclease H1,28,5,12,21,13,27,9,10,17,...,24,17,12,18,8,6,0.000000,0,0,43.11
4,Short coiled-coil protein,10,2,9,12,5,8,2,3,9,...,15,18,6,11,2,2,0.000000,0,0,53.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5951,Gamma-glutamylaminecyclotransferase,14,2,7,12,6,12,7,2,1,...,16,6,5,11,2,7,0.666667,0,1,51.63
5952,Prostamide/prostaglandin F synthase,18,6,12,11,5,22,5,6,11,...,10,13,2,22,2,5,0.666667,0,1,54.11
5953,NEDD8-conjugating enzyme UBE2F,11,5,16,12,7,9,6,6,15,...,11,8,15,12,3,5,0.666667,0,1,53.66
5954,Rab-like protein 2A,16,3,19,18,14,7,5,12,16,...,7,19,13,15,2,9,0.666667,0,1,48.03


# Separo Treino e Teste

Podemos então finalmente separar os dados para treino e teste

In [5]:
TAMANHO_TESTE = 0.2

SEMENTE_ALEATORIA = 42

FEATURES1 = [amino for amino in 'ACDEFGHIKLMNPQRSTVWY']
FEATURES1 += ['Type1 Additive']

FEATURES2 = [amino for amino in 'ACDEFGHIKLMNPQRSTVWY']
FEATURES2 += ['Type2 Additive 1', 'Type2 Additive 2']

TARGET = ["Melting Point (C)"]

### Treino Real

Parte dos dados (5%) é destinado para o teste final, que deverá ser feito apenas uma vez quando a escolha final de atributos da rede e seu treino estejam terminados

In [6]:
indices_treino_real, indices_teste_real = train_test_split(
    df_dados.index, test_size=0.05, random_state=SEMENTE_ALEATORIA
)

df_treino_real = df_dados.loc[indices_treino_real]
df_teste_real = df_dados.loc[indices_teste_real]

x_teste_real1 = df_teste_real.reindex(FEATURES1, axis=1)
x_teste_real2 = df_teste_real.reindex(FEATURES2, axis=1)
y_teste_real = df_teste_real.reindex(TARGET, axis=1)

### Treino Normal

Dos dados restantes (95%), selecionamos 80% para serem utilizados como treino e os 20% (lembrando que ambas as porcentagens são com relação aos 95%) para teste.

In [7]:
indices_treino, indices_teste = train_test_split(
    df_treino_real.index, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df_treino_real.loc[indices_treino]
df_teste = df_treino_real.loc[indices_teste]

x_treino1 = df_treino.reindex(FEATURES1, axis=1)
x_teste1 = df_teste.reindex(FEATURES1, axis=1)

x_treino2 = df_treino.reindex(FEATURES2, axis=1)
x_teste2 = df_teste.reindex(FEATURES2, axis=1)

y_treino = df_treino.reindex(TARGET, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Após a separação dos dados, podemos normaliza-los utilizando algum método de preferência para que se mantenha um padrão para a rede, neste caso, foi selecionado o normalizador de `MinMaxScaler` que gera números de 0 a 1.

In [8]:
normalizador_x1 = StandardScaler()
normalizador_x2 = StandardScaler()
normalizador_y = StandardScaler()

normalizador_x1.fit(x_treino1)
normalizador_x2.fit(x_treino2)
normalizador_y.fit(y_treino)

x_treino_norm1 = normalizador_x1.transform(x_treino1)
x_teste_norm1 = normalizador_x1.transform(x_teste1)

x_treino_norm2 = normalizador_x2.transform(x_treino2)
x_teste_norm2 = normalizador_x2.transform(x_teste2)

y_treino_norm = normalizador_y.transform(y_treino)
y_teste_norm = normalizador_y.transform(y_teste)

Com os dados normalizados, podemos transformá-los em tensores do pytorch para utilizá-los na rede neural.

In [9]:
x_treino_tens1 = torch.tensor(x_treino_norm1, dtype=torch.float32)
x_teste_tens1 = torch.tensor(x_teste_norm1, dtype=torch.float32)

x_treino_tens2 = torch.tensor(x_treino_norm2, dtype=torch.float32)
x_teste_tens2 = torch.tensor(x_teste_norm2, dtype=torch.float32)

y_treino_tens = torch.tensor(y_treino_norm, dtype=torch.float32)
y_teste_tens = torch.tensor(y_teste_norm, dtype=torch.float32)

# Treinamento e teste (não final) das redes

A penúltima etapa do notebook é treinar as redes com os dados e fazer comparações para perceber quais poderiam ser os pontos fortes e fracos destas

### Paramêtros das redes

Mas antes, precisamos definir os parâmetros padrões entre as redes para que seja feita uma comparação justa

In [10]:
c1, c2, c3 = 10, 10, 10
lr, epochs = 0.01, 1000

### MLP Leaky ReLU 1

Aqui, treinamos uma rede que utiliza os dados de input de aditivos do tipo 1 e uma função de ativação ReLU

In [11]:
mlp_leaky_1, rmse_leaky_1 = cria_mlp(torch.nn.LeakyReLU, x_treino_tens1, x_teste_tens1, y_treino_tens, y_teste_tens, normalizador_y, c1, c2, c3, lr, epochs)

Progresso: 100%|█████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 232.95it/s, Perda: 0.5477460623         / Neurons: (10, 10, 10) / LR: 0.01]

Loss do teste: 4.850281715393066


### MLP Sigmoid 1

Aqui, treinamos uma rede que utiliza os dados de input de aditivos do tipo 1 e uma função de ativação Sigmoid

In [12]:
mlp_sigmoid_1, rmse_sigmoid_1 = cria_mlp(torch.nn.Sigmoid, x_treino_tens1, x_teste_tens1, y_treino_tens, y_teste_tens, normalizador_y, c1, c2, c3, lr, epochs)

Progresso: 100%|█████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 273.57it/s, Perda: 0.5514582396         / Neurons: (10, 10, 10) / LR: 0.01]

Loss do teste: 4.858374118804932


### MLP Swish 1

Aqui, treinamos uma rede que utiliza os dados de input de aditivos do tipo 1 e uma função de ativação Swish

In [13]:
mlp_swish_1, rmse_swish_1 = cria_mlp(torch.nn.SiLU, x_treino_tens1, x_teste_tens1, y_treino_tens, y_teste_tens, normalizador_y, c1, c2, c3, lr, epochs)

Progresso: 100%|█████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 234.97it/s, Perda: 0.4149116576         / Neurons: (10, 10, 10) / LR: 0.01]

Loss do teste: 4.4523797035217285


### MLP Leaky ReLU 2

Aqui, treinamos uma rede que utiliza os dados de input de aditivos do tipo 2 e uma função de ativação ReLU

In [14]:
mlp_leaky_2, rmse_leaky_2 = cria_mlp(torch.nn.LeakyReLU, x_treino_tens2, x_teste_tens2, y_treino_tens, y_teste_tens, normalizador_y, c1, c2, c3, lr, epochs)

Progresso: 100%|█████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 232.85it/s, Perda: 0.4784905314         / Neurons: (10, 10, 10) / LR: 0.01]

Loss do teste: 4.6186394691467285


### MLP Sigmoid 2

Aqui, treinamos uma rede que utiliza os dados de input de aditivos do tipo 2 e uma função de ativação Sigmoid

In [15]:
mlp_sigmoid_2, rmse_sigmoid_2 = cria_mlp(torch.nn.Sigmoid, x_treino_tens2, x_teste_tens2, y_treino_tens, y_teste_tens, normalizador_y, c1, c2, c3, lr, epochs)

Progresso: 100%|█████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 262.72it/s, Perda: 0.5025104284         / Neurons: (10, 10, 10) / LR: 0.01]

Loss do teste: 4.702469348907471


### MLP Swish 2

Aqui, treinamos uma rede que utiliza os dados de input de aditivos do tipo 2 e uma função de ativação Swish

In [16]:
mlp_swish_2, rmse_swish_2 = cria_mlp(torch.nn.SiLU, x_treino_tens2, x_teste_tens2, y_treino_tens, y_teste_tens, normalizador_y, c1, c2, c3, lr, epochs)

Progresso: 100%|█████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 229.45it/s, Perda: 0.4412312806         / Neurons: (10, 10, 10) / LR: 0.01]

Loss do teste: 4.572399616241455


Como pudemos ver, a rede que teve melhor desempenho dentro das comparções possíveis foi a que utilizou a função de ativação Leaky ReLU, e por isso, foi considerada para ser utilizada no teste final do projeto.

## Criando os tensores dos dados de teste finais

Para salvar os dados de teste final, precisamos normalizá-los e transformá-los em tensores

In [17]:
x_teste_real_norm1 = normalizador_x1.transform(x_teste_real1)
x_teste_real_norm2 = normalizador_x2.transform(x_teste_real2)

y_teste_real_norm = normalizador_y.transform(y_teste_real)

In [18]:
x_teste_real_tens1 = torch.tensor(x_teste_real_norm1, dtype=torch.float32)
x_teste_real_tens2 = torch.tensor(x_teste_real_norm2, dtype=torch.float32)

y_teste_real_tens = torch.tensor(y_teste_real_norm, dtype=torch.float32)

# Salvando a melhor rede para teste final

A última etapa do notebook é salvar os dados de teste reais para serem utilizados em outro notebook em conjunto com o salvamento da rede neural selecionada para este teste

In [19]:
save = {
    'fn_ativacao': torch.nn.LeakyReLU,
    'x_teste1': x_teste_real_tens1,
    'x_teste2': x_teste_real_tens2,
    'y_teste': y_teste_real_tens,
    'x_norm1': normalizador_x1,
    'x_norm2': normalizador_x2,
    'y_norm': normalizador_y,
    'c1_mlp': c1,
    'c2_mlp': c2,
    'c3_mlp': c3
}

In [20]:
file_data = 'Dados/test_data.pickle'

with open(file_data, 'wb') as file_handler:
    pickle.dump(save, file_handler, protocol=pickle.HIGHEST_PROTOCOL)

file_mlp = 'Dados/mlp_save.pt'

torch.save(mlp_leaky_1.state_dict(), file_mlp)